In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from name_dataset import NameDataset

In [2]:
HIDDEN_SIZE = 100
N_LAYERS = 2
BATCH_SIZE = 256
N_EPOCHS = 100

test_dataset = NameDataset(is_train_set=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=True)


train_dataset = NameDataset(is_train_set=True)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True)

N_COUNTRIES = len(train_dataset.get_countries())
print(N_COUNTRIES, "countries")
N_CHARS = 128 # ASCII

18 countries


In [3]:
def create_variable(tensor):
    if torch.cuda.is_available():
        return tensor.cuda()
    else:
        return tensor

In [4]:
# pad sequences and sort the tensor
def pad_sequences(vectorized_seqs, seq_lengths, countries):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)

    # Sort tensors by their length
    seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]

    # Also sort the target (countries) in the same order
    target = countries2tensor(countries)
    if len(countries):
        target = target[perm_idx]

    # Return variables
    # DataParallel requires everything to be a Variable
    return create_variable(seq_tensor), \
        create_variable(seq_lengths), \
        create_variable(target)


# Create necessary variables, lengths, and target
def make_variables(names, countries):
    sequence_and_length = [str2ascii_arr(name) for name in names]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths, countries)


def str2ascii_arr(msg):
    arr = [ord(c) for c in msg]
    return arr, len(arr)

def countries2tensor(countries):
    country_ids = [train_dataset.get_country_id(
        country) for country in countries]
    return torch.LongTensor(country_ids)

#print(make_variables(['sumin','heh'], ['Korean','Czech'])

In [5]:
class RNNclassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNclassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = int(bidirectional)+1
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, seq_lengths):
        input = input.t()
        batch_size = input.size(1)
        
        hidden = self._init_hidden(batch_size)
        embedded = self.embedding(input)
        gru_input = pack_padded_sequence(embedded, seq_lengths.data.cpu().numpy())
        #self.gru.flatten_parameters()
        output, hidden = self.gru(gru_input, hidden)
        
        return self.fc(hidden[-1])
        
    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers*self.n_directions, batch_size, self.hidden_size)
        return create_variable(hidden)

In [6]:
# Train cycle
def train():
    total_loss = 0

    for i, (names, countries) in enumerate(train_loader, 1):
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)

        loss = criterion(output, target)
        total_loss += loss.data[0]

        classifier.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('[] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                epoch,  i *
                len(names), len(train_loader.dataset),
                100. * i * len(names) / len(train_loader.dataset),
                total_loss / i * len(names)))

    return total_loss


# Testing cycle
def test(name=None):
    # Predict for a given name
    if name:
        input, seq_lengths, target = make_variables([name], [])
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        country_id = pred.cpu().numpy()[0][0]
        print(name, "is", train_dataset.get_country(country_id))
        return

    print("evaluating trained model ...")
    correct = 0
    train_data_size = len(test_loader.dataset)

    for names, countries in test_loader:
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))


In [7]:
if __name__ == '__main__':

    classifier = RNNclassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRIES, N_LAYERS)
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
        classifier = nn.DataParallel(classifier)

    if torch.cuda.is_available():
        classifier.cuda()

    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    #start = time.time()
    print("Training for %d epochs..." % N_EPOCHS)
    for epoch in range(1, N_EPOCHS + 1):
        # Train cycle
        train()

        # Testing
        #test()

        # Testing several samples
        test("Hong")
        test("Choi")

Let's use 2 GPUs!
Training for 100 epochs...


/home/hsue/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/home/hsue/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # Remove the CWD from sys.path while we load stuff.


[] Train Epoch: 1 [2560/13374 (19%)]	Loss: 619.05
[] Train Epoch: 1 [5120/13374 (38%)]	Loss: 538.37
[] Train Epoch: 1 [7680/13374 (57%)]	Loss: 501.92
[] Train Epoch: 1 [10240/13374 (77%)]	Loss: 470.55
[] Train Epoch: 1 [12800/13374 (96%)]	Loss: 447.91
Hong is English
Choi is Russian
[] Train Epoch: 2 [2560/13374 (19%)]	Loss: 321.54
[] Train Epoch: 2 [5120/13374 (38%)]	Loss: 316.36
[] Train Epoch: 2 [7680/13374 (57%)]	Loss: 311.55
[] Train Epoch: 2 [10240/13374 (77%)]	Loss: 309.49
[] Train Epoch: 2 [12800/13374 (96%)]	Loss: 304.20
Hong is English
Choi is Japanese
[] Train Epoch: 3 [2560/13374 (19%)]	Loss: 269.12
[] Train Epoch: 3 [5120/13374 (38%)]	Loss: 267.92
[] Train Epoch: 3 [7680/13374 (57%)]	Loss: 262.45
[] Train Epoch: 3 [10240/13374 (77%)]	Loss: 256.73
[] Train Epoch: 3 [12800/13374 (96%)]	Loss: 251.66
Hong is English
Choi is Japanese
[] Train Epoch: 4 [2560/13374 (19%)]	Loss: 229.42
[] Train Epoch: 4 [5120/13374 (38%)]	Loss: 221.36
[] Train Epoch: 4 [7680/13374 (57%)]	Loss: 215

[] Train Epoch: 30 [5120/13374 (38%)]	Loss: 19.65
[] Train Epoch: 30 [7680/13374 (57%)]	Loss: 20.68
[] Train Epoch: 30 [10240/13374 (77%)]	Loss: 20.97
[] Train Epoch: 30 [12800/13374 (96%)]	Loss: 21.93
Hong is Chinese
Choi is Korean
[] Train Epoch: 31 [2560/13374 (19%)]	Loss: 19.96
[] Train Epoch: 31 [5120/13374 (38%)]	Loss: 21.01
[] Train Epoch: 31 [7680/13374 (57%)]	Loss: 21.18
[] Train Epoch: 31 [10240/13374 (77%)]	Loss: 21.78
[] Train Epoch: 31 [12800/13374 (96%)]	Loss: 22.29
Hong is Chinese
Choi is Korean
[] Train Epoch: 32 [2560/13374 (19%)]	Loss: 22.29
[] Train Epoch: 32 [5120/13374 (38%)]	Loss: 21.39
[] Train Epoch: 32 [7680/13374 (57%)]	Loss: 19.89
[] Train Epoch: 32 [10240/13374 (77%)]	Loss: 20.94
[] Train Epoch: 32 [12800/13374 (96%)]	Loss: 20.76
Hong is Chinese
Choi is Korean
[] Train Epoch: 33 [2560/13374 (19%)]	Loss: 17.62
[] Train Epoch: 33 [5120/13374 (38%)]	Loss: 17.80
[] Train Epoch: 33 [7680/13374 (57%)]	Loss: 18.46
[] Train Epoch: 33 [10240/13374 (77%)]	Loss: 18.88


[] Train Epoch: 59 [7680/13374 (57%)]	Loss: 10.58
[] Train Epoch: 59 [10240/13374 (77%)]	Loss: 10.62
[] Train Epoch: 59 [12800/13374 (96%)]	Loss: 10.75
Hong is Korean
Choi is Korean
[] Train Epoch: 60 [2560/13374 (19%)]	Loss: 8.79
[] Train Epoch: 60 [5120/13374 (38%)]	Loss: 9.30
[] Train Epoch: 60 [7680/13374 (57%)]	Loss: 9.89
[] Train Epoch: 60 [10240/13374 (77%)]	Loss: 10.33
[] Train Epoch: 60 [12800/13374 (96%)]	Loss: 11.01
Hong is Korean
Choi is Korean
[] Train Epoch: 61 [2560/13374 (19%)]	Loss: 7.77
[] Train Epoch: 61 [5120/13374 (38%)]	Loss: 8.27
[] Train Epoch: 61 [7680/13374 (57%)]	Loss: 9.08
[] Train Epoch: 61 [10240/13374 (77%)]	Loss: 10.25
[] Train Epoch: 61 [12800/13374 (96%)]	Loss: 11.05
Hong is Korean
Choi is Korean
[] Train Epoch: 62 [2560/13374 (19%)]	Loss: 12.95
[] Train Epoch: 62 [5120/13374 (38%)]	Loss: 14.89
[] Train Epoch: 62 [7680/13374 (57%)]	Loss: 15.21
[] Train Epoch: 62 [10240/13374 (77%)]	Loss: 15.91
[] Train Epoch: 62 [12800/13374 (96%)]	Loss: 16.79
Hong is 

[] Train Epoch: 89 [2560/13374 (19%)]	Loss: 8.08
[] Train Epoch: 89 [5120/13374 (38%)]	Loss: 7.94
[] Train Epoch: 89 [7680/13374 (57%)]	Loss: 8.37
[] Train Epoch: 89 [10240/13374 (77%)]	Loss: 9.08
[] Train Epoch: 89 [12800/13374 (96%)]	Loss: 9.21
Hong is Korean
Choi is Korean
[] Train Epoch: 90 [2560/13374 (19%)]	Loss: 8.88
[] Train Epoch: 90 [5120/13374 (38%)]	Loss: 8.37
[] Train Epoch: 90 [7680/13374 (57%)]	Loss: 9.18
[] Train Epoch: 90 [10240/13374 (77%)]	Loss: 9.17
[] Train Epoch: 90 [12800/13374 (96%)]	Loss: 9.55
Hong is Chinese
Choi is Korean
[] Train Epoch: 91 [2560/13374 (19%)]	Loss: 8.48
[] Train Epoch: 91 [5120/13374 (38%)]	Loss: 8.29
[] Train Epoch: 91 [7680/13374 (57%)]	Loss: 8.41
[] Train Epoch: 91 [10240/13374 (77%)]	Loss: 9.09
[] Train Epoch: 91 [12800/13374 (96%)]	Loss: 9.17
Hong is Chinese
Choi is Korean
[] Train Epoch: 92 [2560/13374 (19%)]	Loss: 9.14
[] Train Epoch: 92 [5120/13374 (38%)]	Loss: 8.38
[] Train Epoch: 92 [7680/13374 (57%)]	Loss: 8.56
[] Train Epoch: 92 [